In [0]:
#imports 

from pyspark.sql.functions import current_timestamp

In [0]:
#Variaveis do Unity Catalog
CATALOGO = "agroclima"
SCHEMA = "silver"
TABELA = "municipios"

# Montar caminho da tabela
caminho_tabela = f"{CATALOGO}.{SCHEMA}.{TABELA}"

In [0]:
# Carrega a tabela de munipios 
df_municipios = spark.read.table("agroclima.bronze.municipios")

In [0]:
# Carrega Tabela de Localização usando o Metodo sql e ja selecionando as colunas que serão usadas
df_localizacao = spark.sql("""
                           
                           SELECT 
                                 codigo_ibge,
                                 ddd,
                                 latitude,
                                 longitude,
                                 siafi_id
                           FROM 
                                 agroclima.bronze.localizacao_municipios
                    """)

#### Transformação

In [0]:
#Verifica Nome das Colunas do dataframe
df_municipios.columns

In [0]:
# Renmeia Colunas do Dataframe
df_transformado = df_municipios.withColumnRenamed("id","municipio_id") \
                        .withColumnRenamed("microrregiao.id", "microrregiao_id") \
                        .withColumnRenamed("microrregiao.nome", "microrregiao") \
                        .withColumnRenamed("microrregiao.mesorregiao.id", "mesorregiao_id") \
                        .withColumnRenamed("microrregiao.mesorregiao.nome", "mesorregiao") \
                        .withColumnRenamed("microrregiao.mesorregiao.UF.id", "uf_id") \
                        .withColumnRenamed("microrregiao.mesorregiao.UF.nome", "estado") \
                        .withColumnRenamed("microrregiao.mesorregiao.UF.regiao.id", "regiao_id") \
                        .withColumnRenamed("microrregiao.mesorregiao.UF.regiao.nome", "regiao") \
                        .withColumnRenamed("microrregiao.mesorregiao.UF.regiao.sigla", "regiao_sigla") \
                        .withColumnRenamed("regiao-imediata.id", "regiao_imediata_id") \
                        .withColumnRenamed("regiao-imediata.nome", "regiao_imediata") \
                        .withColumnRenamed("regiao-imediata.regiao-intermediaria.id","regiao_intermediaria_id")\
                        .withColumnRenamed("regiao-imediata.regiao-intermediaria.nome", "regiao_intermediaria")

In [0]:
# Enriquecimento da Base juntando os dados de municipio com a localização
df_transformado = df_transformado.join(df_localizacao, df_transformado.municipio_id == df_localizacao.codigo_ibge, "left")

#display(df_transformado)

#### LImpeza

In [0]:
#Lista de Colunas que se~rao removidas
colunas_a_remover = ["microrregiao.mesorregiao.UF.sigla", "regiao-imediata.regiao-intermediaria.UF.id", \
                     "regiao-imediata.regiao-intermediaria.UF.sigla", "regiao-imediata.regiao-intermediaria.UF.nome", "regiao-imediata.regiao-intermediaria.UF.regiao.id", "regiao-imediata.regiao-intermediaria.UF.regiao.sigla", "regiao-imediata.regiao-intermediaria.UF.regiao.nome", "codigo_ibge"]

In [0]:
# Remove Colunas que não serão utilizadas
df_limpeza = df_transformado.drop(*colunas_a_remover)

In [0]:
# Verificando se há linhas duplicadas 
#print(df_limpeza.count())
#print(df_limpeza.distinct().count())

#### Carga

In [0]:
#Adiciona data da ingestão dos dados
df_carga = df_limpeza.withColumn('_data_ingestao', current_timestamp())

In [0]:
df_carga.write.format("delta") \
              .mode('overwrite') \
              .partitionBy("uf") \
              .option("overwriteSchema", "true") \
              .saveAsTable(caminho_tabela)